In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.special as spe
from hansolo import *
import psutil

# sklearn includes a complement of datasets which can be used
# to explore different types of machine learning examples
from sklearn import datasets
from sklearn.model_selection import train_test_split

#retrieve dataset
digits = datasets.load_digits()
images=digits.images/16 #(1797,8,8)
target=digits.target #(1797,)

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)
M_train = np.shape(X_train)[0]
M_test = np.shape(X_test)[0]

# Reshape images and compute probabilities
flattened_images_train = X_train.reshape((M_train, -1))
flattened_images_test = X_test.reshape((M_test, -1))

M0 = 40 #number of images to train hidden layer
M0_sel = 40 #number of images to select hidden neurons
M_out = M_train - M0 - M0_sel #number of images to train output neurons
M_tot = M0 + M0_sel + M_out + M_test

N = 2000 #number of time steps
p = 1  # spiking proba input neurons when pixel is black
nu = 0.5 # bias

K_input = 64 #number of input neurons
K_mid = int(K_input * (K_input - 1)/2) #number of hidden neurons before selection
#K_mid_new = 200

K_output = 10  #number of output neurons
eta_mid = 3 #parameter EWA hidden neurons
expert1 = 'EWA' #expert hidden layer
para_PWA = 2 #parameter PWA 

expert2 = 'EWA' #expert output layer

indices = np.arange(K_input)
i, j = np.meshgrid(indices, indices)
mask = i < j
l_mid = np.column_stack((i[mask], j[mask])) #tensor indices hidden neurons

eta_output = 0.007 #parameter EWA output neurons

param = 0.7 #param to renormalize input neurons gains for output
param2 = 0.25 #param to renormalize output activity

images_mid = flattened_images_train[:M0+M0_sel,:] /16 #images to train mid neurons
images_out = flattened_images_train[M0+M0_sel:M0+M0_sel+M_out,:] /16 #images to train output neurons
images_test = flattened_images_test /16 #test images

target_out = y_train[M_train - M_out:] #labels of train images
target_test = y_test #labels of test images
Nb_simu = 10 #number of simulations

accuracy = np.zeros((Nb_simu))

for K_mid_new in 10* np.arange(1,10): #number of selected neurons hidden layerS
    print(K_mid_new)
    K_tot = K_mid_new + K_input
    for nb in range(Nb_simu):
    #Simulation input neurons
        prob =  np.transpose(np.array([p*images_mid for i in range(N)]), axes = (2,0,1))

    # Input Neurons activity (K_input,N,M)
        input_neurons_mid = stats.bernoulli.rvs(prob) #to train mid neurons

    #to train output neurons

        prob_input = np.transpose(np.array([p*images_out for i in range(N)]), axes = (2,0,1))
        input_neurons_out = stats.bernoulli.rvs(prob_input)

    #to test
        prob_input = np.transpose(np.array([p*images_test for i in range(N)]), axes = (2,0,1))
        input_neurons_test = stats.bernoulli.rvs(prob_input) #to test

    #training mid neurons
        
        cum_gain_EWA = np.zeros((K_mid, K_input))
        for m in range(M0):
            gain = GainMid(np.arange(K_mid),K_mid,K_input,input_neurons_mid[:,:,m], l_mid)
            cum_gain_EWA += gain

        if expert1 == 'EWA':
            W_mid_end = spe.softmax(eta_mid * cum_gain_EWA, axis = 1)

    #selection mid neurons

    #simulation of mid neurons activity to select
        prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid_end, input_neurons_mid[:, :, M0:M0 + M0_sel])
        clipped_probas_mid = np.clip(prob_mid, 0, 1)
        mid_neurons_sel = stats.bernoulli.rvs(clipped_probas_mid)

    #selection
        P_mid = np.sum(mid_neurons_sel, axis=1) / N
        max_P = np.max(P_mid, axis = 1 )
        max_P_sorted = np.sort(max_P)

        x = max_P_sorted[K_mid-K_mid_new] 
        mid_neur_ok = np.where(max_P > x)[0] #selected neurons
        missing = K_mid_new - np.shape(mid_neur_ok)[0]
        mid_neur_to_sel = np.where(max_P == x)[0]
        n_tot = np.shape(mid_neur_to_sel)[0]
        selected_indices = np.random.choice(n_tot, size=missing, replace=False) #random choice of neurons which have the same nb of spikes
        mid_neur_missing = mid_neur_to_sel[selected_indices]

        mid_neur = np.concatenate((mid_neur_missing, mid_neur_ok))
        mid_neur = np.sort(mid_neur)

        prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid_end[mid_neur,:], input_neurons_out)
        clipped_probas_mid = np.clip(prob_mid, 0, 1)
        mid_neurons_out=stats.bernoulli.rvs(clipped_probas_mid)

        P_mid = np.sum(mid_neurons_out, axis = 1) / N
        P_input = np.sum(input_neurons_out, axis = 1) / N

    
    #training output
        if expert2 == 'EWA':
            cum_gain_EWA = np.zeros((K_output, K_tot))
            for m in range(M_out-1):
                gain = GainOutputAllConnected(K_output, K_mid_new, K_input, param, P_mid[:,m], P_input[:,m], target_out[m])
                cum_gain_EWA += gain
            W_output_end = spe.softmax(eta_output * cum_gain_EWA, axis = 1)

    #test with new images

        prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid_end[mid_neur,:], input_neurons_test)
        clipped_probas_mid = np.clip(prob_mid, 0, 1)
        mid_neurons_test=stats.bernoulli.rvs(clipped_probas_mid)
        answers_test = np.zeros((M_test))

        for m in range(M_test):
            activity_tot = np.concatenate((mid_neurons_test[:,:,m], param2 * input_neurons_test[:,:,m]), axis = 0) 
        # Simulation output neurons 
            probas_output = np.sum(W_output_end[:,None,:] * activity_tot.T, axis = 2)
            probas_output = np.clip(probas_output, 0, 1)
            output_neurons = stats.bernoulli.rvs(probas_output)

            P_output = np.sum(output_neurons, axis = 1) / N
            chosen_class = np.where(P_output == np.max(P_output))[0]

            if chosen_class[0] == target_test[m]:
                answers_test[m] = 1
        accuracy[nb] = np.sum(answers_test)/M_test
        print(nb, accuracy[nb])

    #np.save(f"accuracy_all_{expert1}_{expert2}_{K_mid_new}_{eta_output}_10", accuracy)